Numpy Basics
---------------------

Numpy is the universal standard for working with numerical data in Python, and it’s at the core of the scientific Python and PyData ecosystems. In this tutorial we will understand the basic concepts of working with numpy as a user. 

## Tabular Data Container

Numpy is meant to handle structured or tabular data. In numpy, this is represented as an <a href="https://numpy.org/doc/stable/reference/arrays.ndarray.html">ndarray</a>, or n-dimensional array. You can create an ndarray in various ways

* Read data from file, e.g. using ``np.loadtxt`` or ``np.genfromtxt`` methods
* From python list and tuples
* Preset values, like 0 or 1

In [1]:
import numpy as np

# From python lists
x = np.array([100, 200, 300])

# From constant
y = np.ones(shape=[3])
z = np.ones_like(y)

x + y + z

array([102., 202., 302.])

## Properties of a Numpy Array

A numpy array is composed of 4 main attributes:

* ``data`` is the underlying data in the array lives in a contiguous, one dimensional byte array or buffer in memory. It is in row-major format but can also be column-major if you explicitly specify.
* ``shape`` contains the number of elements of the array along each dimension.
* ``dtype`` data types of the elements.
* ``strides`` number of steps to take in the internal buffer to go the next element along a specific dimension.

In [2]:
x = np.arange(1000, 1100)
x.data, x.shape, x.dtype, x.strides

(<memory at 0x7f68929aaf40>, (100,), dtype('int64'), (8,))

## Numpy dtype hierarchy

<img src="https://numpy.org/doc/stable/_images/dtype-hierarchy.png" />

## Array Indexing

Individual elements can be accessed or updated using the indexing operator.

In [3]:
# 1D
print(x[0], x[:5], x[-1])

1000 [1000 1001 1002 1003 1004] 1099


In [4]:
# 2D
x = np.array([[100, 200], [50, 70]])
print(x[0, 0], x[0, :], x[:, 1])

100 [100 200] [200  70]


In [5]:
# Update one element
x[0, 0] = 2
x

array([[  2, 200],
       [ 50,  70]])

In [6]:
# Update along one dimension
x[0, :] = [10, 20]
x

array([[10, 20],
       [50, 70]])

In [7]:
# Indexing with Booleans
x = np.arange(1, 10)
odd_index = [i % 2 == 0 for i in x]
x[odd_index]

array([2, 4, 6, 8])

In numpy, array slices usually do not require a memory copy. The ``data`` of the slice is actually a "view" of the data of the original ndarray. We can check this using the ``base`` attribute or the ``flags.owndata`` attribute. 

In [8]:
x = np.array([[1, 2], [3, 4]])
y = x[0, :]

print(x.base is None, id(y.base) == id(x)) 

True True


In [9]:
y.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

By default, updating the slice also updates the owner's data. This can sometimes lead to accidental updates in multiple arrays sharing the underlying data.

In [10]:
y[0] = 100
print(y)
print(x)

[100   2]
[[100   2]
 [  3   4]]


There is no default "copy on write" mechanism, so we need to manually copy the slice to prevent the update at source. Alternatively,
we can set ``flags.writeable`` to False.

In [11]:
y.flags.writeable = False
y[0] = 200

ValueError: assignment destination is read-only

## Iterating over Arrays

This is usually an anti-pattern. Most common use-cases are covered by numpy library functions which are internally vectorized. For the same reason, incrementally extending a numpy array is also an anti-pattern. Instead, you should first allocate an array of the target shape and then update it in-place.

The built-in "for" loop always iterates over the first dimension

In [12]:
x = np.array([[1, 2], [3, 4]])

# Row-wise sum of x
for idx, row in enumerate(x):
    print(f"Sum of row {idx} is {sum(row)}")

Sum of row 0 is 3
Sum of row 1 is 7


For more complicated use-cases:
* <a href="https://numpy.org/doc/stable/reference/generated/numpy.ndindex.html">np.ndindex</a> generates n-dimension index values.

* <a href="https://numpy.org/doc/stable/reference/arrays.nditer.html">np.nditer<a/> is useful to iterate over one or more arrays element by element.
    
Note that neither of these methods are particularly useful in iterating over arrays along arbitrary dimensions. There's also no performance improvement over transposing the array and using the built-in for loop.

In [13]:
x = np.array([[1, 2], [3, 4]])

# row wise sum of x
print("Row wise sums:")
for row_index in np.ndindex(x.shape[0]):
    print(sum(x[row_index]))

print("Column wise sums:")
# column wise sum of x
# Here we need to transpose x appropriately, so the x[col_index] returns the first column
# also works for > 2 dimensions
for col_index in np.ndindex(x.shape[1]):
    print(sum(x.T[col_index]))

Row wise sums:
3
7
Column wise sums:
4
6


## Common Use Cases

### Creating Arrays

In [14]:
# Create array of zeros
np.zeros([2, 2])

array([[0., 0.],
       [0., 0.]])

In [15]:
# Array of ones
np.ones([2, 2])

array([[1., 1.],
       [1., 1.]])

In [16]:
# Array of specific value
np.full([2, 2], 10)

array([[10, 10],
       [10, 10]])

In [17]:
# Allocated but not filled
np.empty([2, 2])

array([[1., 1.],
       [1., 1.]])

In [18]:
# Similar to full
np.repeat(10, 5)

array([10, 10, 10, 10, 10])

In [19]:
# Can also repeat arrays
np.repeat([1, 2, 3, 4], 5)

array([1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4])

In [20]:
np.tile([[1, 2], [3, 4]], 4)

array([[1, 2, 1, 2, 1, 2, 1, 2],
       [3, 4, 3, 4, 3, 4, 3, 4]])

In [21]:
### Adding Dimensions
x = np.array([[1, 2], [3, 4]]).reshape(4)
x

array([1, 2, 3, 4])

In [22]:
x.reshape([1, 4])

array([[1, 2, 3, 4]])

In [23]:
x = np.arange(1, 10)
np.atleast_2d(x)

array([[1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [24]:
x = np.arange(1, 10)
x = x[:, np.newaxis]
x

array([[1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [25]:
# Removing dimension
x = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9]])
np.squeeze(x)

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

### Selecting Values

In [26]:
# Select values according to some condition
x = np.arange(10)
x[x > 5]

array([6, 7, 8, 9])

In [27]:
# Update values where conditions are met
x = np.arange(1, 10)
np.where(x > 5, x, 0)

array([0, 0, 0, 0, 0, 6, 7, 8, 9])

In [28]:
# Get multiple values by index - 1D
x = np.array([1, 2, 3, 4])
x[[1, 2]]

array([2, 3])

In [29]:
x.take([1, 2])

array([2, 3])

In [30]:
# Get multiple values by index - 2D
x = np.array([[1, 2, 3], [4, 5, 6]])

# we want to get the elements at index (0, 0) and (1, 1)
x[[0, 1], [0, 1]]

array([1, 5])

In [31]:
# multiple values in condition

x = np.arange(1, 10)
x[np.isin(x, [4, 16])]

array([4])

### Aggregate Functions

In [32]:
# By default aggrgate functions work on the entire array 

x = np.array([[1, 2, 3], [4, 5, 6]])
x.mean(), x.sum(), x.std(), x.mean()

(3.5, 21, 1.707825127659933, 3.5)

In [33]:
# We can also specify which axis

x = np.array([[1, 2, 3], [4, 5, 6]])
x.mean(axis=0)

array([2.5, 3.5, 4.5])

In [34]:
## Elementwise Functions

x = np.arange(0, 180, 30)
print(x)
print(np.sin(np.radians(x)))

[  0  30  60  90 120 150]
[0.        0.5       0.8660254 1.        0.8660254 0.5      ]


### Matrix And Linear Algebra

In [35]:
# dot product of two vectors
np.dot([1, 2, 3], [1, 2, 3])

14

In [36]:
# Matrix multiplication
np.matmul([[1, 2, 3]], [[1], [2], [3]])

array([[14]])

In [37]:
# identity matrix
np.eye(5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [38]:
# inverse of a matrix
np.linalg.inv(np.eye(5))

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [-0., -0., -0.,  1., -0.],
       [ 0.,  0.,  0.,  0.,  1.]])

In [39]:
# solve system of linear equations
# x0 + 2 * x1 = 1 and 3 * x0 + 5 * x1 = 2

A = [[1, 2], [3, 5]]
b = [1, 2]

# Solve Ax = b
np.linalg.solve(A, b)

array([-1.,  1.])

In [40]:
# Eigenvalues and EigenVectors

x = np.diag((1, 2, 3))
x

array([[1, 0, 0],
       [0, 2, 0],
       [0, 0, 3]])

In [41]:
evals, evecs = np.linalg.eig(x)
evals, evecs

(array([1., 2., 3.]),
 array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]))

## Structure and Record Arrays

In some applications where we have 2D data, we have to execute a lot of column related operations, e.g. what are the average sales every month by region? In these situation, it's not nice to work directly with numpy arrays. In the above example, we have to remember the column index for the value we want to aggregate. Numpy structure arrays solve this problem.

In [42]:
sales = np.array(
    [
        (100, 'jan', 'uk'), 
        (270, 'jan', 'eu'),
        (120, 'feb', 'uk'), 
        (250, 'feb', 'eu'),
    ],
    dtype=[('qty', 'int'), ('month', 'U3'), ('region', 'U2')]
)

sales = sales.view(np.recarray)

In [43]:
sales

rec.array([(100, 'jan', 'uk'), (270, 'jan', 'eu'), (120, 'feb', 'uk'),
           (250, 'feb', 'eu')],
          dtype=[('qty', '<i8'), ('month', '<U3'), ('region', '<U2')])

In [44]:
# What are the jan sales in EU region?
jan_eu_filter = np.logical_and(sales.region == 'eu', sales.month == 'jan')
sales[jan_eu_filter].qty

array([270])

## Working with Missing Data

In real life, working with missing data is often a prerequisite. Most of the numpy data reading methods replace missing data with ``np.nan``. This is a special value in the IEEE 754 floating point specification. The other special value is ``np.inf``. Use ``-np.inf`` for negative infinity.

Most of the numpy functions do not handle ``np.nan`` or ``np.inf`` well. For example,

In [45]:
x = np.array([10, 1, 5, np.nan, 2.5])
np.max(x)

nan

The general pattern for processing 1D arrays like these are

1. Remove the nan & infinity instances
2. Execute numpy ops
3. If required, put back the nan & infinity instances.

In [48]:
# note that nan != nan, so we have to use the special nan handling method
x = x[~np.isnan(x)]
np.max(x)

10.0

In [50]:
# Use np.isfinite() to keep only finite, non-nan data points
x = np.array([10, 1, 5, np.nan, np.inf, 2.5])
np.max(x[np.isfinite(x)])

10.0

This strategy doesn't generalize to 2D or higher dimensional data. Every row in a 2D array might contain different number of nans. Removing them will result in a jagged array which doesn't fit the numpy tabular structure. This results in a lot of application specific code.

In [56]:
x = np.array([
    [1.0, 2.0, np.nan, np.nan],
    [np.nan, 1.0, 3.0, 4.0]
])

# max for every row in x
print(np.max(x, axis=1))

def nansafe_max(row):
    return np.max(row[np.isfinite(row)])

np.apply_along_axis(nansafe_max, 1, x)

[nan nan]


array([2., 4.])

In some cases, there are nansafe versions of numpy functions. For example:

In [58]:
x = np.array([
    [1.0, 2.0, np.nan, np.nan],
    [np.nan, 1.0, 3.0, 4.0]
])
np.nanmax(x, axis=1)

array([2., 4.])

Another alternative is to use the <a href="https://numpy.org/doc/stable/reference/maskedarray.html">masked array</a> feature of numpy. This is a more flexible and generic way of handling missing and invalid values.

In [69]:
import numpy.ma as ma

x = [-1, 1, 2, -1, 3, 1, 0, 2, np.nan, np.inf]

# (-1, nan, inf) values are invalid 
cond = np.logical_or(~np.isfinite(x), np.isin(x, [-1]))
m_x = ma.masked_where(cond, x)

m_x

masked_array(data=[--, 1.0, 2.0, --, 3.0, 1.0, 0.0, 2.0, --, --],
             mask=[ True, False, False,  True, False, False, False, False,
                    True,  True],
       fill_value=1e+20)

In [70]:
ma.mean(m_x)

1.5

## Broadcast

This is a feature of numpy that comes into play in arithmetic operations involving multiple numpy arrays. When one of the arrays is different in shape from the other arrays, numpy can automatically adjust smaller array so that it is compatible with the larger array in shape. This happens in the C layer of numpy, without making data copies, allowing efficient vectorized implementation.

<img src="https://jakevdp.github.io/PythonDataScienceHandbook/figures/02.05-broadcasting.png">

## Resources

* <a href="https://bedford-computing.co.uk/learning/wp-content/uploads/2015/10/Python-for-Data-Analysis.pdf">Python for Data Analysis by Wes McKinney</a>

* <a href="https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf">numpy cheat sheet</a>